In [ ]:
# default_exp core

# module name here

> 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from collections import OrderedDict
from typing import Union, Tuple, Sequence, Set
from numpy.random import RandomState
import numpy as np
from datasets import Dataset
from datasets import load_dataset
from pathlib import Path
from sklearn.model_selection import StratifiedShuffleSplit
from torchvision.transforms import (CenterCrop, 
                                    RandomErasing,
                                    RandomAutocontrast,
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    RandomAdjustSharpness,
                                    ToTensor)
import torch
from transformers import AutoFeatureExtractor, TrainingArguments, Trainer
from transformers import AutoModelForImageClassification
from datasets import load_metric
from rich import print

testing

In [ ]:
from numpy.testing import assert_allclose
from toolz.dicttoolz import valmap
from collections import Counter
from toolz import frequencies

## Data loading

In [ ]:
ds = load_dataset("davanstrien/flysheet", use_auth_token=True, streaming=False, split='train')

Using custom data configuration davanstrien--flysheet-2cdc8849e04b41c9
Reusing dataset parquet (/Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [ ]:
f =  '/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/or_5268_fse002r/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/or_5268_fse002r (1).jpg.jpg'

In [ ]:
f

'/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/or_5268_fse002r/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/or_5268_fse002r (1).jpg.jpg'

In [ ]:
import re

In [ ]:
f = re.sub(r"(\(\d\))","",f)

In [ ]:
f.split('.')[0]

'/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/or_5268_fse002r/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/or_5268_fse002r '

In [ ]:
def return_base_path_deduplicated(x):
    f = x['fpath']
    f = re.sub(r"(\(\d\))","",f)
    f = f.split(".")[0]
    f = f.rstrip()
    return {"clean_path": re.sub(r"(\(\d\))","",f)}

In [ ]:
def check_uniques(example, uniques, column='clean_path'):
    if example[column] in uniques:
        uniques.remove(example[column])
        return True
    else:
        return False

In [ ]:
ds

Dataset({
    features: ['image', 'label', 'fpath'],
    num_rows: 2061
})

In [ ]:
def drop_duplicates(ds):
    ds = ds.map(return_base_path_deduplicated)
    uniques = set(ds['clean_path'])
    ds = ds.filter(check_uniques, fn_kwargs={"uniques":uniques})
    return ds

In [ ]:
ds = drop_duplicates(ds)

0ex [00:00, ?ex/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
def get_id(example):
    x = example["fpath"]
    x = Path(x).name.split("_")
    return {"id": "_".join(x[:2] if len(x) >= 3 else x[:3])}

In [ ]:
ds = ds.map(get_id)

0ex [00:00, ?ex/s]

In [ ]:
ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x248 at 0x19DE98940>,
 'label': 0,
 'fpath': '/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/add_ms_10455_fse005r.jpg',
 'clean_path': '/Users/dvanstrien/Documents/DS/hmd_flysheet_detection/data/Flysheet_data/CONTAINER/add_ms_10455_fse005r',
 'id': 'add_ms'}

In [ ]:
ds

Dataset({
    features: ['image', 'label', 'fpath', 'clean_path', 'id'],
    num_rows: 1223
})

## Train, valid, test splits

In [ ]:
#export
def split_w_stratify(
    ds,
    test_size: Union[int, float],
    train_size: Union[int, float, None] = None,
    random_state: Union[int, RandomState, None] = None,
) -> Tuple[Dataset, Dataset]:
    labels = ds['label']
    label_array = np.array(labels)
    train_inds, valid_inds = next(
        StratifiedShuffleSplit(
            n_splits=2, test_size=test_size, random_state=random_state
        ).split(np.zeros(len(labels)), y=label_array)
    )
    return ds.select(train_inds), ds.select(valid_inds)

In [ ]:
train, valid = split_w_stratify(ds, test_size=0.5)

test frequencies 

In [ ]:
assert_allclose(train.shape, valid.shape,rtol=2)

In [ ]:
train_freqs = frequencies(train['label'])
train_freqs

{5: 25, 2: 108, 3: 93, 0: 36, 6: 139, 1: 30, 4: 164, 7: 16}

In [ ]:
train_percentages =  OrderedDict(sorted(valmap(lambda x: x/len(train_freqs),train_freqs).items())).values()
train_percentages

odict_values([4.5, 3.75, 13.5, 11.625, 20.5, 3.125, 17.375, 2.0])

In [ ]:
valid_freqs = frequencies(valid['label'])
valid_percentages = OrderedDict(sorted(valmap(lambda x: x/len(valid_freqs),valid_freqs).items())).values()
valid_percentages

odict_values([4.5, 3.625, 13.625, 11.75, 20.625, 3.125, 17.375, 1.875])

In [ ]:
assert_allclose(list(train_percentages), list(valid_percentages), atol=1)

In [ ]:
#export
def train_valid_split_w_stratify(
    ds,
    valid_size: Union[int,float]=None,
    test_size: Union[int, float]=0.3,
    train_size: Union[int, float, None] = None,
    random_state: Union[int, RandomState, None] = None,
) -> Tuple[Dataset,Dataset, Dataset]:
    train, valid_test = split_w_stratify(ds, test_size=test_size)
    valid, test = split_w_stratify(valid_test, test_size=test_size)
    return train, valid, test

In [ ]:
train, valid, test = train_valid_split_w_stratify(ds)

In [ ]:
def prepare_dataset(ds):
    print("Preparing dataset...")
    print("dropping duplicates...")
    ds = drop_duplicates(ds)
    print("getting ID...")
    ds = ds.map(get_id)    
    print("creating train, valid, test splits...")
    train, valid, test = train_valid_split_w_stratify(ds)
    data = {"train": train, 
            "valid": valid, 
            "test": test}
    for k,v  in data.items():
        print(f"{k} has {len(v)} examples")
    return train,valid,test

In [ ]:
ds = load_dataset("davanstrien/flysheet", use_auth_token=True, streaming=False, split='train')

Using custom data configuration davanstrien--flysheet-2cdc8849e04b41c9
Reusing dataset parquet (/Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


In [ ]:
train,valid,test = prepare_dataset(ds)
train,valid,test

Preparing dataset...

dropping duplicates...

Loading cached processed dataset at /Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-1dfba86c98cf68e0.arrow
Loading cached processed dataset at /Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-f34fde9b42e1bc61.arrow


getting ID...

Loading cached processed dataset at /Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-3c87c3e3b7dda5fa.arrow


creating train, valid, test splits...

train has 856 examples

valid has 256 examples

test has 111 examples

(Dataset({
     features: ['image', 'label', 'fpath', 'clean_path', 'id'],
     num_rows: 856
 }),
 Dataset({
     features: ['image', 'label', 'fpath', 'clean_path', 'id'],
     num_rows: 256
 }),
 Dataset({
     features: ['image', 'label', 'fpath', 'clean_path', 'id'],
     num_rows: 111
 }))

## Augmentations 

In [ ]:
model_checkpoint = "davanstrien/vit-base-patch16-224-in21k-base-manuscripts"

In [ ]:
#export
def prepare_transforms(model_checkpoint, train_ds, valid_ds, test_ds=None):
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
    normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
    _train_transforms = Compose(
            [
                Resize((feature_extractor.size,feature_extractor.size)),
                RandomAdjustSharpness(0.1),
                RandomAutocontrast(),
                ToTensor(),
                normalize,
                RandomErasing()
            ]
        )

    _val_transforms = Compose(
            [
                Resize((feature_extractor.size, feature_extractor.size)),
                ToTensor(),
                normalize,
            ]
        )

    def train_transforms(examples):
        examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
        return examples

    def val_transforms(examples):
        examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
        return examples
    if test_ds is not None:
        test_ds.set_transform(val_transforms)
    train_ds.set_transform(train_transforms)
    valid_ds.set_transform(val_transforms)
    return train_ds, valid_ds, test_ds

In [ ]:
train_ds, valid_ds, test_ds = prepare_transforms(model_checkpoint, train,valid, test)

In [ ]:
train_ds[0]['pixel_values'].shape

torch.Size([3, 224, 224])

In [ ]:
#export
def prep_data(ds_checkpoint="davanstrien/flysheet", model_checkpoint=None):
    ds = load_dataset(ds_checkpoint, use_auth_token=True, streaming=False, split='train')
    labels = ds.info.features['label'].names
    id2label = dict(enumerate(labels))
    label2id = {v:k for k,v in id2label.items()}
    train,valid, test = prepare_dataset(ds)
    train_ds, valid_ds, test_ds = prepare_transforms(model_checkpoint, train, valid, test)
    return train_ds, valid_ds, test_ds, id2label, label2id

## Model training 

In [ ]:
#export
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
from torch.optim import AdamW
import transformers



In [ ]:
def train_model(ds_checkpoint, model_checkpoint, num_epochs, save_dir,tune=False):
    transformers.logging.set_verbosity_warning()
    train_ds, valid_ds, test_ds, id2label, label2id = load_data(ds_checkpoint,model_checkpoint=model_checkpoint)
    model = AutoModelForImageClassification.from_pretrained(model_checkpoint, num_labels=len(id2label),
                                                   id2label=id2label,
                                                  label2id=label2id, ignore_mismatched_sizes=True)
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
    # if tune:
    #     disable_tqdm = True
    # else:
    #     disable_tqdm = False
    args = TrainingArguments(
    f"save_dir/{model_checkpoint}_flyswot",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    push_to_hub=False,
    learning_rate=2e-5,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    num_train_epochs=num_epochs,
    weight_decay=0.1,disable_tqdm=False,
    fp16=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='logs',
    remove_unused_columns=False,
    save_total_limit=10,
        optim=AdamW,
    seed=666,    
)
    f1 = load_metric("f1")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return f1.compute(predictions=predictions, references=labels, average='macro')


    trainer = Trainer(model,
                      args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor)
    trainer.train()
    return trainer

In [ ]:
trainer = train_model('davanstrien/flysheet', "facebook/deit-tiny-patch16-224",0.1,'test',)

Using custom data configuration davanstrien--flysheet-2cdc8849e04b41c9
Reusing dataset parquet (/Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Preparing dataset...

dropping duplicates...

Loading cached processed dataset at /Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-1dfba86c98cf68e0.arrow
Loading cached processed dataset at /Users/dvanstrien/.cache/huggingface/datasets/parquet/davanstrien--flysheet-2cdc8849e04b41c9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-f34fde9b42e1bc61.arrow


getting ID...

...

creating train, valid, test splits...

train has 856 examples

valid has 256 examples

test has 111 examples

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([8, 192]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model management

## Model Evaluation 

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

extractor = AutoFeatureExtractor.from_pretrained("davanstrien/convnext-tiny-224_flyswot")

model = AutoModelForImageClassification.from_pretrained("davanstrien/convnext-tiny-224_flyswot")

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/951 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/106M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline('image-classification',model=model,feature_extractor=extractor)